In [1]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point
import altair as alt

alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
geojson_path = "Communities-Chicago.geojson"
with open(geojson_path, 'r') as f:
    communities_geojson = json.load(f)

In [3]:
# Load and preprocess CSV data
complaints_path = 'CDPH_Environmental_Complaints.csv'
complaints_df = pd.read_csv(complaints_path)

# Lowercase the 'COMPLAINT TYPE' column
complaints_df['COMPLAINT TYPE'] = complaints_df['COMPLAINT TYPE'].str.lower()

# Convert 'COMPLAINT DATE' to a datetime object and categorize decades
complaints_df['COMPLAINT DATE'] = pd.to_datetime(complaints_df['COMPLAINT DATE'])
complaints_df.dropna(subset=['COMPLAINT DATE'], inplace=True)
# Extract the year from 'COMPLAINT DATE' and store it in a new column 'COMPLAINT YEAR'
complaints_df['COMPLAINT YEAR'] = complaints_df['COMPLAINT DATE'].dt.year

# Extract latitude and longitude from the 'LOCATION' column
complaints_df['LATITUDE'] = complaints_df['LOCATION'].str.extract(r'POINT \((.*?) (.*?)\)')[1].astype(float)
complaints_df['LONGITUDE'] = complaints_df['LOCATION'].str.extract(r'POINT \((.*?) (.*?)\)')[0].astype(float)

# Load community geometries
communities_gdf = gpd.read_file(geojson_path)

# Create a GeoDataFrame with point geometries for complaints
geometry = [Point(xy) for xy in zip(complaints_df['LONGITUDE'], complaints_df['LATITUDE'])]
complaints_gdf = gpd.GeoDataFrame(complaints_df, geometry=geometry)

# Perform a spatial join to associate complaints with neighborhoods
complaints_with_neighborhood = gpd.sjoin(complaints_gdf, communities_gdf, how='left', op='within')

complaints_df = complaints_with_neighborhood[['COMPLAINT TYPE', 'COMPLAINT YEAR', 'LATITUDE', 'LONGITUDE', 'area_num_1', 'INSPECTOR']]

print(complaints_df.head(15))

                               COMPLAINT TYPE  COMPLAINT YEAR   LATITUDE  \
0                             noise complaint            1994  41.882436   
1                    air pollution work order            2008  41.882841   
2                             noise complaint            1996  41.883306   
3                    air pollution work order            1996  41.883341   
4                    air pollution work order            1995  41.909701   
5   service stations/storage tanks work order            2000  41.979658   
6                    air pollution work order            1995  41.868770   
7                    air pollution work order            1994  41.741261   
8                  illegal dumping work order            2000  41.709820   
9                         asbestos work order            1995  41.869520   
10                                      other            1993  41.707960   
11                            noise complaint            1993  41.705268   
12          

C:\Users\Arka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\Arka\AppData\Local\Temp\ipykernel_29632\809161435.py:26: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  complaints_with_neighborhood = gpd.sjoin(complaints_gdf, communities_gdf, how='left', op='within')


In [4]:
communities_gdf = gpd.read_file(geojson_path)
communities_gdf.crs = "EPSG:4326"

# Calculate the centroids
centroids = communities_gdf['geometry'].centroid

# Extract the latitude and longitude and add them as new columns
communities_gdf['latitude_centroid'] = centroids.y
communities_gdf['longitude_centroid'] = centroids.x

communities_gdf = communities_gdf[['area_num_1', 'community', 'latitude_centroid', 'longitude_centroid']]
print(communities_gdf)

   area_num_1        community  latitude_centroid  longitude_centroid
0          35          DOUGLAS          41.835118          -87.618678
1          36          OAKLAND          41.823750          -87.603216
2          37      FULLER PARK          41.809085          -87.632425
3          38  GRAND BOULEVARD          41.812949          -87.617860
4          39          KENWOOD          41.808916          -87.596184
..        ...              ...                ...                 ...
72         74  MOUNT GREENWOOD          41.694879          -87.713192
73         75      MORGAN PARK          41.689730          -87.669054
74         76            OHARE          41.975684          -87.893701
75         77        EDGEWATER          41.986712          -87.663417
76          9      EDISON PARK          42.007613          -87.813781

[77 rows x 4 columns]


C:\Users\Arka\AppData\Local\Temp\ipykernel_29632\1798080765.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = communities_gdf['geometry'].centroid


In [5]:
# Merge the two dataframes on the 'area_num_1' column
merged_df = complaints_df.merge(communities_gdf, on='area_num_1', how='left')

# Print the merged dataframe
merged_df = merged_df.dropna()
print(merged_df)

                    COMPLAINT TYPE  COMPLAINT YEAR   LATITUDE  LONGITUDE  \
0                  noise complaint            1994  41.882436 -87.626829   
1         air pollution work order            2008  41.882841 -87.662404   
2                  noise complaint            1996  41.883306 -87.627969   
3         air pollution work order            1996  41.883341 -87.652663   
4         air pollution work order            1995  41.909701 -87.653183   
...                            ...             ...        ...        ...   
57541              noise complaint            2022  41.881096 -87.641998   
57542  construction and demolition            2023  41.699623 -87.626664   
57543     air pollution work order            2023  41.713027 -87.557534   
57544              noise complaint            2022  42.017369 -87.668867   
57545  construction and demolition            2023  41.942554 -87.757848   

      area_num_1 INSPECTOR        community  latitude_centroid  \
0             32     

In [6]:
import altair as alt

# Calculate the minimum and maximum latitude and longitude values
min_latitude = merged_df['latitude_centroid'].min()
max_latitude = merged_df['latitude_centroid'].max()
min_longitude = merged_df['longitude_centroid'].min()
max_longitude = merged_df['longitude_centroid'].max()

# Define a selection
select_point = alt.selection_single(empty='all', on='click', fields=['latitude_centroid', 'longitude_centroid'])

# Define a selection for the line chart
select_inspector = alt.selection_single(fields=['INSPECTOR'], empty='all', on='click')

# Create a scatter plot with scale limits and link the selection
scatter_plot = alt.Chart(merged_df).mark_circle(size=200).encode(
    x=alt.X('longitude_centroid:Q', scale=alt.Scale(domain=[min_longitude, max_longitude])),
    y=alt.Y('latitude_centroid:Q', scale=alt.Scale(domain=[min_latitude, max_latitude])),
    tooltip=['community:N'],
).add_selection(
    select_point
).properties(
    width=700,
    height=500,
    title='Scatter Plot of Complaints by Location'
)

pie_data = merged_df.groupby(['COMPLAINT TYPE', 'latitude_centroid', 'longitude_centroid'])['COMPLAINT TYPE'].count().reset_index(name='COUNT')

# Define a pie chart for complaints based on the selection
pie_chart = alt.Chart(pie_data).mark_arc().encode(
    theta='COUNT:Q',
    color=alt.Color('COMPLAINT TYPE:N', scale=alt.Scale(scheme='category20'), legend=alt.Legend(title='Category Legend', orient='right')),
    tooltip=['COMPLAINT TYPE:N', 'COUNT:Q']
).transform_filter(
    select_point
).properties(
    width=400,
    height=400,
    title='Pie Chart of Complaints'
)

# Combine the scatter plot and pie chart and link them
scatter_pie_chart = scatter_plot | pie_chart
scatter_pie_chart = scatter_pie_chart.configure_legend(
    orient='top-right',
    titleLimit=0,
    offset=-180,
)

# Display the chart
scatter_pie_chart.display()

with open("linked_charts/linked_scatter_pie/linked_scatter_pie_raw_files.json", "w") as f:
    f.write(json.dumps(scatter_pie_chart.to_dict(), indent=4))

# with open("../JSON/linked_scatter_pie_merged_df.json", "w") as f:
#     f.write(merged_df.to_json(indent=4))

# with open("../JSON/linked_scatter_pie_inspector_pie_data.json", "w") as f:
#     f.write(pie_data.to_json(indent=4))

C:\Users\Arka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
C:\Users\Arka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\altair\vegalite\v5\api.py:394: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
C:\Users\Arka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.HConcatChart(...)